In [1]:
!pip install --upgrade pip
!pip install beautifulsoup4

Requirement already up-to-date: pip in /opt/conda/lib/python3.6/site-packages (18.0)


## wikipedia 한국어 

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import csv
from urllib import parse

def loadPage(baseUrl, path, targetDir, depth, indexWriter, linkset):
    if path in linkset :
        return
    
    linkset.add(path)
    
    url = baseUrl + path
    print("loading ", url)
    
    try :
        html = urlopen(url)
        bsObj = BeautifulSoup(html, "html.parser")
        title = bsObj.find("h1", {"id":"firstHeading"}).text

        contentsDiv = bsObj.find("div", {"id":"mw-content-text"})
        contentsDiv = contentsDiv.find("div", {"class":"mw-parser-output"})
        
        paras = contentsDiv.findAll({"h1","h2","h3","h4","h5","h6","p"})
        
        links = []
        linkTitles = []
        for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a", 
            href=re.compile("^(/wiki/)((?!:).)*$")):
              if 'href' in link.attrs:
                  links.append(link.attrs['href'])
                  linkTitles.append(link.text)

        filename = parse.quote(title)
        filename = filename.replace('/','_') + ".txt"
        filepath = targetDir + "/" + filename

        indexWriter.writerow([title, filename])
        
        with open(filepath, 'w', encoding='utf-8') as file :
            file.write(title)
            file.write("\n\n")
            
            
            for l in linkTitles:
                file.write(l)
                file.write("\n")
                
            for p in paras:
                if p.name == 'p':
                    file.write(p.text)
                else:
                    p = p.find("span",{"class":"mw-headline"})
                    if p:
                        file.write(p.text)
                file.write("\n")
            
                
            file.close()

        if(depth > 0):
            depth -= 1
            for link in links:
                loadPage(baseUrl, link, targetDir, depth, indexWriter, linkset)
    except Exception as ex :  
        print(ex)
    
        

# 경제사상사 : /wiki/%EA%B2%BD%EC%A0%9C%EC%82%AC%EC%83%81%EC%82%AC
economicThoughts = "/wiki/%EA%B2%BD%EC%A0%9C%EC%82%AC%EC%83%81%EC%82%AC"
# 경제학 : /wiki/%EA%B2%BD%EC%A0%9C%ED%95%99
economics = "/wiki/%EA%B2%BD%EC%A0%9C%ED%95%99"
socialogy = "/wiki/%EC%82%AC%ED%9A%8C%ED%95%99"
# 은닉마르코프 모델 : /wiki/%EC%9D%80%EB%8B%89_%EB%A7%88%EB%A5%B4%EC%BD%94%ED%94%84_%EB%AA%A8%EB%8D%B8
markov = "/wiki/%EC%9D%80%EB%8B%89_%EB%A7%88%EB%A5%B4%EC%BD%94%ED%94%84_%EB%AA%A8%EB%8D%B8"
# 체계이론 : /wiki/%EB%B6%84%EB%A5%98:%EC%B2%B4%EA%B3%84%EC%9D%B4%EB%A1%A0
system = "/wiki/%EB%B6%84%EB%A5%98:%EC%B2%B4%EA%B3%84%EC%9D%B4%EB%A1%A0"
# 보험
insurance = "/wiki/%EB%B3%B4%ED%97%98"

data_dir = "../../data/wiki2"

with open("../../data/wiki_page_index.csv", 'w', encoding='utf-8') as file :
    indexWriter = csv.writer(file)
    linkset = set()
    loadPage("https://ko.wikipedia.org", economics, 
             data_dir, 1, indexWriter, linkset)
    file.close()

loading  https://ko.wikipedia.org/wiki/%EA%B2%BD%EC%A0%9C%ED%95%99
loading  https://ko.wikipedia.org/wiki/%EC%95%84%ED%94%84%EB%A6%AC%EC%B9%B4%EC%9D%98_%EA%B2%BD%EC%A0%9C
loading  https://ko.wikipedia.org/wiki/%EC%95%84%EC%8B%9C%EC%95%84%EC%9D%98_%EA%B2%BD%EC%A0%9C
loading  https://ko.wikipedia.org/wiki/%EB%AF%B8%EC%8B%9C%EA%B2%BD%EC%A0%9C%ED%95%99
loading  https://ko.wikipedia.org/wiki/%EA%B1%B0%EC%8B%9C%EA%B2%BD%EC%A0%9C%ED%95%99
loading  https://ko.wikipedia.org/wiki/%EA%B2%BD%EC%A0%9C%EC%82%AC%EC%83%81%EC%82%AC
loading  https://ko.wikipedia.org/wiki/%EA%B3%84%EB%9F%89%EA%B2%BD%EC%A0%9C%ED%95%99
loading  https://ko.wikipedia.org/wiki/%EC%8B%A4%ED%97%98%EA%B2%BD%EC%A0%9C%ED%95%99
loading  https://ko.wikipedia.org/wiki/%ED%96%89%EB%8F%99%EA%B2%BD%EC%A0%9C%ED%95%99
loading  https://ko.wikipedia.org/wiki/%EB%AC%B8%ED%99%94%EA%B2%BD%EC%A0%9C%ED%95%99
loading  https://ko.wikipedia.org/wiki/%EA%B2%BD%EC%A0%9C%EC%84%B1%EC%9E%A5
loading  https://ko.wikipedia.org/wiki/%EA%B0%9C%EB%B0%9C%EA%B2

In [3]:
import os
import sys
sys.path.insert(0, '../')
import pycor

model_size = 7000
outputpath = "../../output/" + str(model_size) + "/"
model_path = outputpath + "model/"

def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = listfiles(data_dir)


print(pycor.version())
pycor.loadmodel(model_path)

Init Trainer
Init DocResolver
0.0.5
Loading Model from ../../output/7000/model/
Load  ../../output/7000/model//heads.csv   소요시간: 1.453
Load  ../../output/7000/model//tails.csv   소요시간: 0.013


In [4]:
_,words_array = pycor.readfile(files[1])

pycor.printSentences(words_array)


게마인자메 노름다타이 

독일어 
영어 
독일 국립도서관 
전거 파일 
전거 통제 
VIAF 
게마인자메 노름다타이 ( 독일어 : Gemeinsame Normdatei 영어 : Integrated Authority File GND ) 는 독일 국립도서관이 운영하는 전거 파일이다 . 


GND 엔티티 종류 
GND에는 7개의 주요 엔티티가 있다 : 
[ 1 ] 

각주 
외부 링크 


In [5]:
words_array, tags_array = pycor.__trim(words_array)

for words in words_array:
    print(words)

['게마인자메', '노름다타이']
['독일어']
['영']
['독일', '국립도서관']
['전거', '파일']
['전거', '통제']
['VIAF']
['게마인자메', '노름다타이', '독일어', 'GEMEINSAME', 'NORMDATEI', '영', 'INTEGRATED', 'AUTHORITY', 'FILE', 'GND', '', '독일', '국립도서관', '운영하', '전거', '파일']
['GND', '엔티티', '종류']
['GND', '7개', '주요', '엔티티', '있']
['1']
['각주']
['외부', '링크']


In [6]:

from pycor.res import WikiResolver

wikiResolver = WikiResolver(pycor.getmodel())

pycor.setresolver(wikiResolver)

_,words_array = pycor.readfile(files[1])

keywords = wikiResolver.resolveKeywords(words_array)

for word in keywords:
    head = word.bestpair.head
    print(head.text, head.pos)

Init WikiResolver
독일어 {'C'}
영어 {'C'}
VIAF {'C'}
각주 {'C'}
